In [1]:
path = "/Users/mac/Desktop/763pj/data/"

In [2]:
import cv2
import numpy as np
import pandas as pd
import random
import pickle
import os
from PIL import Image
from os.path import isfile
from IPython.display import Image, display
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
model_path = path + "CNN_NN.model"

BOX_SIZE = 512

def preprocess_image(img):
    from keras.applications.densenet import preprocess_input
    return preprocess_input(img)


def get_branch_model(inp_shape):
    from keras.applications.densenet import DenseNet121
    model = DenseNet121(input_shape=inp_shape, include_top=False, weights=None, pooling='max')
    return model

def build_model(lr, l2, activation='sigmoid'):

    ##############
    # BRANCH MODEL
    ##############
    regul  = regularizers.l2(l2)
    optim  = Adam(lr=lr)
    kwargs = {'padding':'same', 'kernel_regularizer':regul}


    inp = Input(shape=img_shape) # 384x384x1
    x   = Conv2D(64//down_size, (9,9), strides=2, activation='relu', **kwargs)(inp)

    x   = MaxPooling2D((2, 2), strides=(2, 2))(x) # 96x96x64
    for _ in range(2):
        x = BatchNormalization()(x)
        x = Conv2D(64//down_size, (3,3), activation='relu', **kwargs)(x)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 48x48x64
    x = BatchNormalization()(x)
    x = Conv2D(128//down_size, (1,1), activation='relu', **kwargs)(x) # 48x48x128
    for _ in range(2): x = subblock(x, 64//down_size, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 24x24x128
    x = BatchNormalization()(x)
    x = Conv2D(256//down_size, (1,1), activation='relu', **kwargs)(x) # 24x24x256
    for _ in range(2): x = subblock(x, 64//down_size, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 12x12x256
    x = BatchNormalization()(x)
    x = Conv2D(384//down_size, (1,1), activation='relu', **kwargs)(x) # 12x12x384
    for _ in range(2): x = subblock(x, 96//down_size, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 6x6x384
    x = BatchNormalization()(x)
    x = Conv2D(512//down_size, (1,1), activation='relu', **kwargs)(x) # 6x6x512
    for _ in range(2): x = subblock(x, 128//down_size, **kwargs)
    
    x             = GlobalMaxPooling2D()(x) # 512
    branch_model  = Model(inp, x)
    
    ############
    # HEAD MODEL
    ############
    mid        = 32
    xa_inp     = Input(shape=branch_model.output_shape[1:])
    xb_inp     = Input(shape=branch_model.output_shape[1:])
    x1        = Lambda(lambda x : K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x2         = Lambda(lambda x : K.square(x))(x1)
    x          = Concatenate()([x1, x2])
    x          = Reshape((2, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x          = Conv2D(1, (2, 1), activation='relu', padding='valid')(x)
    x          = Reshape((branch_model.output_shape[1], 1, 1))(x)
    x          = Flatten(name='flatten')(x)
    
    x          = Dense(32, use_bias=True, activation=activation)(x)
    x          = Dense(1, use_bias=True, activation=activation)(x)
    head_model = Model([xa_inp, xb_inp], x, name='head')

    ########################
    # SIAMESE NEURAL NETWORK
    ########################
    img_a      = Input(shape=img_shape)
    img_b      = Input(shape=img_shape)
    xa         = branch_model(img_a)
    xb         = branch_model(img_b)
    x          = head_model([xa, xb])
    model      = Model([img_a, img_b], x)
    model.compile(optim, loss='binary_crossentropy', metrics=['binary_crossentropy', 'acc'])
    return model, branch_model, head_model

In [4]:

def get_trained_model():
    print(00)
    model, branch_model, head_model = build_model(img_shape=(BOX_SIZE, BOX_SIZE, 3))
    print(00)
    model.load_weights(model_path)
    return model, branch_model, head_model


def expand_path(p):
    if isfile(path+'train/' + p):
        return path+'train/' + p
    return p


def get_boxes():
    temp_p2bb = pickle.load(open(path + 'bounding_box.pickle', 'rb'))
    p2bb = {}
    for k in temp_p2bb:
        p2bb[k] = temp_p2bb[k]
    return p2bb


def read_single_image(path):
    try:
        img = np.array(Image.open(path))
    except:
        try:
            img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        except:
            print('Fail')
            return None

    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    if img.shape[2] == 2:
        img = img[:, :, :1]

    if img.shape[2] == 1:
        img = np.concatenate((img, img, img), axis=2)

    if img.shape[2] > 3:
        img = img[:, :, :3]

    return img


def read_cropped_image(p, x0, y0, x1, y1, img_shape=(224, 224, 3)):
    anisotropy = 2.15
    crop_margin = 0.05

    # Read the image
    img = read_single_image(p)
    size_x, size_y = img.shape[1], img.shape[0]

    dx = x1 - x0
    dy = y1 - y0
    x0 -= dx * crop_margin
    x1 += dx * crop_margin + 1
    y0 -= dy * crop_margin
    y1 += dy * crop_margin + 1
    if x0 < 0: x0 = 0
    if x1 > size_x: x1 = size_x
    if y0 < 0: y0 = 0
    if y1 > size_y: y1 = size_y
    dx = x1 - x0
    dy = y1 - y0
    if dx > dy * anisotropy:
        dy = 0.5 * (dx / anisotropy - dy)
        y0 -= dy
        y1 += dy
    else:
        dx = 0.5 * (dy * anisotropy - dx)
        x0 -= dx
        x1 += dx
    
    if x0 < 0: x0 = 0
    if x1 > size_x: x1 = size_x
    if y0 < 0: y0 = 0
    if y1 > size_y: y1 = size_y
    x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)

    if y0 != y1 and x0 != x1:
        img = img[y0:y1, x0:x1, :]
    img = cv2.resize(img, (img_shape[1], img_shape[0]), interpolation=cv2.INTER_LINEAR)
    if len(img.shape) == 2:
        img = np.concatenate((img, img, img), axis=2)

    return img


def normalize_array(arr):
    arr = 255.0 * (arr - arr.min()) / (arr.max() - arr.min())
    return arr

In [5]:


def create_activation_map_for_images(model, images, preproc_image):
    images_preproc = preproc_image(images.astype(np.float32))
    preds = model.predict(images_preproc)
    print('Shape of predictions: {}'.format(preds.shape))

    hmaps = []
    for z in range(images.shape[0]):
        img_orig = images[z]
        heatmap = preds[z]
    
        # Uncomment it to emulate RELU activation
        # heatmap[heatmap < 0] = 0.

        ch0 = np.zeros_like(heatmap[:, :, 0])
        ch1 = np.zeros_like(heatmap[:, :, 0])
        ch2 = np.zeros_like(heatmap[:, :, 0])

        # Find how often maximum is in each pixel.
        for k in range(heatmap.shape[2]):
            p = heatmap[:, :, k]
            mx = p.max()
            if mx == 0:
                continue
            for i in range(heatmap.shape[0]):
                for j in range(heatmap.shape[1]):
                    if p[i, j] == mx:
                        ch0[i, j] += 1
                        ch2[i, j] += 1  

        for i in range(heatmap.shape[0]):
            for j in range(heatmap.shape[1]):
                mn = heatmap[i, j].min()
                mx = heatmap[i, j].max()
                mean = heatmap[i, j].mean()
                std = heatmap[i, j].std()
                # print(i, j, mn, mx, mean, std, mx - mn)
                ch1[i, j] = std

        ch0 = normalize_array(ch0)
        ch1 = normalize_array(ch1)
        ch2 = normalize_array(ch2)
        ch = np.stack((ch0, ch1, ch2), axis=2)

        ch = cv2.resize(ch.astype(np.uint8), (img_orig.shape[1], img_orig.shape[0]), interpolation=cv2.INTER_LANCZOS4)

        ch = normalize_array(ch)

        heat = (0.2 * img_orig + (0.1 * img_orig * ch) / 255 + 0.7 * ch).astype(np.uint8)
        heat = heat.astype(np.uint8)
        heat = normalize_array(heat)

        img_line = np.concatenate(
            (img_orig, ch, heat), axis=1
        )
        hmaps.append(img_line)

    return np.array(hmaps)



In [ ]:
print(11)
from keras.models import Model
print(21)
model, branch_model, head_model = get_trained_model()
print(0)

bboxes = get_boxes()
print(1)
x = branch_model.layers[-3].output
branch_model_modified = Model(inputs=branch_model.inputs, outputs=x)

start = 10
end = 20
image_ids = os.listdir(path+'train')[start:end]
images = []
for image_id in image_ids:
    print(image_id)
    f = expand_path(image_id)
    bb = bboxes[image_id]
    img_orig = read_cropped_image(f, bb[0], bb[1], bb[2], bb[3], img_shape=(BOX_SIZE, BOX_SIZE, 3))
    images.append(img_orig)

img_line = create_activation_map_for_images(branch_model_modified, np.array(images), preprocess_image)

for i, image_id in enumerate(image_ids):
    print(i,"!!")
    img = cv2.cvtColor(img_line[i].astype(np.uint8), cv2.COLOR_RGB2BGR)
    img_path = image_id
    cv2.imwrite(img_path, img)
    print('Heatmap for image: {}'.format(image_id))
    # Image(img_line[i].astype(np.uint8))
    display(Image(filename=img_path))